<a href="https://colab.research.google.com/github/mschuessler/two4two/blob/trainKerasExample/examples/two4two_leNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pathlib
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
from google.colab import drive


# Two4two data training
The notebook demonstrates how to train a modern LeNet CNN on a Dataset pregenerated with the [two4two Module](https://github.com/mschuessler/two4two).

If you open this notebook in Colab please make sure to request a GPU Instance. Training times will be excessively slow otherwise.

# Mounting Google drive to save trained model later
Since colab is a free resource it runtimes are limited. Hence we want to save our model to reuse it late when our reserved instance terminates. To do this we mount a google drive. If you don't want to use Google drive or if you run this notebook on your own juypter notebook server please skip the next cell.

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


We will now define the path to which or trained model will be saved. You may alter the name of the dictionary according to your preference. If you run this notebook outside of Google Colab just change this path to a local path of your notebook server.

In [9]:
model_filepath = "/content/gdrive/My Drive/two4two_example_model"

We will use the the callback functionality of keras to save our model whenever we achieved a new highest validation accuracy when training the model.

In [10]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Defining model architecture / loading previously trained model
The follwoing cell test wheter the dictionary where the model should be saved exist, if so we try to load it.

If the directory does not exist we define a new model according to a modern LeNet architecture and complie it using the ADAM optimizer.

In [5]:
trained_model_exists = os.path.exists(model_filepath)

In [11]:
if trained_model_exists:
  modernLenetModel = keras.models.load_model(model_filepath)
else:
  modernLenetModel = keras.models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(2, activation="softmax"),
    ])
  modernLenetModel.compile(loss="categorical_crossentropy",
                             optimizer="adam", metrics=["accuracy"])
    

# Download example dataset
In this example we will be using a pregenerated dataset package. It contains a larger dataset called "medVarSpherObjColorBias". This dataset contains two biases. The sphercitiy of the blocks as well as their color are somewhat predictive of the label peaky or stretchy. The package contains 40.000 images for training, 10.000 for validation and 1.000 for testing.

It also contains the testing data of 1000 images each for three other datasets that contain only one or none of the two biases.

Hence, we would expect that if we train a model on the dataset with two biases it should before worse on the other test sets, if the biases are used by the model for its predictions.

In [12]:
datasets = ["medVarSpherObjColorBias", "medVar",	"medVarObjColorBias", "medVarSpherBias"]

In [13]:
dataset_url = data_dir = keras.utils.get_file(
    origin="https://f001.backblazeb2.com/file/two4two/datasets_models/example_dataset.tar.gz",
    fname="example_dataset",
    untar=True
)

1214414848/1214411699 [==============================] - 45s 0us/step


# Reading dataframe from jsonl

In [14]:
train_dir = os.path.join(data_dir, datasets[0], "train")
train_df = pd.read_json(os.path.join(train_dir, "parameters.jsonl"), lines=True)
train_df["filename"] = train_df["id"] + ".png"

In [15]:
valid_dir = os.path.join(data_dir, datasets[0], "validation")
valid_df = pd.read_json(os.path.join(valid_dir, "parameters.jsonl"), lines=True)
valid_df["filename"] = valid_df["id"] + ".png"

# Creating Datagenerator from dataframes

In [16]:
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_dataframe(dataframe=train_df, directory=train_dir,
                                              x_col="filename", y_col="obj_name", batch_size=64)
valid_generator = datagen.flow_from_dataframe(dataframe=valid_df, directory=valid_dir,
                                              x_col="filename", y_col="obj_name", batch_size=64)
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

Found 40000 validated image filenames belonging to 2 classes.
Found 10000 validated image filenames belonging to 2 classes.


# Train Model
We highly recommend to train the model at least for 30 or even better 45 epochs.

In [17]:
modernLenetModel.fit(train_generator,
                     steps_per_epoch=STEP_SIZE_TRAIN,
                     validation_data=valid_generator,
                     validation_steps=STEP_SIZE_VALID,
                     epochs=45,
                     callbacks = [model_checkpoint_callback]
                     )

Epoch 1/45
625/625 [==============================] - 152s 189ms/step - loss: 0.4403 - accuracy: 0.7981 - val_loss: 0.4407 - val_accuracy: 0.7989
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/two4two_example_model/assets
Epoch 2/45
625/625 [==============================] - 115s 183ms/step - loss: 0.3900 - accuracy: 0.8226 - val_loss: 0.3754 - val_accuracy: 0.8212
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/two4two_example_model/assets
Epoch 3/45
625/625 [==============================] - 114s 182ms/step - loss: 0.3557 - accuracy: 0.8403 - val_loss: 0.3204 - val_accuracy: 0.8587
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/two4two_example_model/assets
Epoch 4/45
625/625 [==============================] - 114s 182ms/step - loss: 0.3362 - accuracy: 0.8512 - val_loss: 0.2994 - val_accuracy: 0.8698
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/two4two_example_model/assets
Epoch 5/45
625/625 [==============================] - 114s 1

# Evaluating the model
We now use our trained model on the test sets of all datasets. As expected the model perfroms worse on the models were not all biases are present.

In [19]:
for dataset_name in datasets:
        test_dir = os.path.join(data_dir, dataset_name, "test")
        test_df = pd.read_json(os.path.join(test_dir, "parameters.jsonl"), lines=True)
        test_df["filename"] = test_df["id"] + ".png"

        datagen = ImageDataGenerator(rescale=1. / 255)
        test_generator = datagen.flow_from_dataframe(dataframe=test_df, directory=test_dir,
                                                     x_col="filename", y_col="obj_name",
                                                     batch_size=64)

        print("Evaljateing on " + dataset_name)
        modernLenetModel.evaluate(test_generator)[1]

Found 1000 validated image filenames belonging to 2 classes.
Evaljateing on medVarSpherObjColorBias
16/16 [==============================] - 3s 197ms/step - loss: 0.2185 - accuracy: 0.9110
Found 1000 validated image filenames belonging to 2 classes.
Evaljateing on medVar
16/16 [==============================] - 3s 164ms/step - loss: 0.4988 - accuracy: 0.8100
Found 1000 validated image filenames belonging to 2 classes.
Evaljateing on medVarObjColorBias
16/16 [==============================] - 3s 167ms/step - loss: 0.5059 - accuracy: 0.8040
Found 1000 validated image filenames belonging to 2 classes.
Evaljateing on medVarSpherBias
16/16 [==============================] - 3s 167ms/step - loss: 0.2185 - accuracy: 0.9110
